In [1]:
import sys
import os 
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

In [2]:
from data_attacks.alg1 import alg1_0 as a1
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from models import mnist

In [3]:
transform = transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=50,num_workers=6)

testset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size=50, num_workers=6)

In [4]:
net2 = mnist.MNIST2()
net2.load_state_dict(torch.load('../models/mnist-net2.pt'))
net2.eval()

MNIST2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [6]:
figure = plt.figure(figsize=(8, 8))
id = 0
img, label = testset[id]
target = 3
newImg, eps = a1.attack(net2,img,target)
figure.add_subplot(1, 2, 1)
plt.axis("off")
plt.imshow(img.squeeze(), cmap="gray")
figure.add_subplot(1, 2, 2)
plt.axis("off")
plt.imshow(newImg.squeeze(), cmap="gray")
plt.show()

AttributeError: module 'torch' has no attribute 'tensorenp'

<Figure size 576x576 with 0 Axes>